---



# <font color='#33ffa3'>**Análisis Multimodal de Síntomas en la Enfermedad de Parkinson**</font>

In [21]:
#@title **Importar librerías**

# Manipulación de datos
import os
import cv2
import numpy as np
import pandas as pd
from zipfile import ZipFile


# Utils
import matplotlib.pyplot as plt


# sklearn
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import PCA

In [11]:
#@title **Cargamos los datos desde Drive**

from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/data_parkinson')
print(os.getcwd())

ValueError: mount failed

In [5]:
#@title **Extraer el zip**

os.mkdir('../../../../data')
os.popen('cp data.csv ../../../../data/data.csv')
target_path = os.path.join('../../../../data') # Aquí es donde van a quedar los datos
# deben quedar fuera de drive para poder acceder a ellos 'localmente'

# Unzip the downloaded files
zip_file_path = os.path.join('AudioVisualData_v7.zip')
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_path)

# # Me muevo hasta donde quedaron los datos
%cd ../../../../data
# Verifico
%ls

# <font color='#33ffe7'>**1. Procesamiento del dataset**</font>

## <font color='#52F17F'>**Creación del Dataset Pandas auxiliar**</font>

In [6]:
data = pd.read_csv("data.csv", index_col='patient')

def string2array(string: str):
  """ Convierte un string con datos a numpy array """
  string = string.strip('[]')
  return np.fromstring(string, sep=',')

data['flatten_log_mel_spectogram'] = data['flatten_log_mel_spectogram'].apply(string2array)
data.head()

In [7]:
"""
Constante sacada de la media de cantidad de frames de todos los vídeos multiplicada por 0.1
para tener el 10%
"""

FRAMES_PROMEDIO = 14

def preprocess_frame(frame_path: str):
  frame = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE) # Leemos la imagen desde la ruta en escala de grises
  # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # La pasamos de BGR a RGB
  frame = cv2.resize(frame, (112, 112)) # Resize a 112x112
  frame = frame / 255. # Normalizamos entre 0 y 1
  return frame.flatten()
  # return frame

def preprocess_video(video_path: str, n_frames = FRAMES_PROMEDIO):
  frames = []
  for frame in sorted(os.listdir(video_path))[:n_frames]:
    frames.append(preprocess_frame(f"{video_path}/{frame}"))
  return np.concatenate(frames)
  # return frames

# Preprocesar todos los audios y almacenar los espectrogramas mel
data['flatten_frame_sequence'] = data['frames_path'].apply(preprocess_video)

# Verificar los datos
data.head()

In [8]:
def preprocess_and_fuse(flatten_frame_sequence, flatten_log_mel_spectogram):
    """
    Esta función concatena los frames aplanados y el espectrograma mel aplanado.

    Args:
        flatten_frame_sequence: array de frames aplanados.
        flatten_log_mel_spectogram: array de espectrograma mel aplanado.

    Returns:
        fused_features: Un array que contiene la concatenación de los frames y el espectrograma mel.
    """
    # Concatenar los frames y el espectrograma mel
    return np.concatenate([flatten_frame_sequence, flatten_log_mel_spectogram])

In [9]:
# Fusionar todas las características aplanadas
data['fused_flattened_features'] = data.apply(
    lambda row: preprocess_and_fuse(row['flatten_frame_sequence'], row['flatten_log_mel_spectogram']),
    axis=1
)

# Verificar los datos
data.head()

## <font color='#52F17F'>**PCA Analysis**</font>

### <font color='#EB9A54'>**Hipomimia**</font>

In [12]:
pca_video = PCA()
pca_video.fit(np.vstack(data['flatten_frame_sequence']))

# Varianza explicada acumulada
explained_variance_ratio = np.cumsum(pca_video.explained_variance_ratio_)

# Graficar la varianza explicada acumulada
plt.figure(figsize=(10, 6))
plt.plot(explained_variance_ratio, marker='o', linestyle='--')
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulada')
plt.title('Varianza explicada acumulada por número de componentes principales')
plt.grid(True)
plt.show()

# Imprimir el número de componentes para diferentes niveles de varianza explicada
thresholds = [0.90, 0.95, 0.99]
for threshold in thresholds:
    num_components = np.argmax(explained_variance_ratio >= threshold) + 1
    print(f'Número de componentes para retener el {int(threshold*100)}% de la varianza: {num_components}')

### <font color='#EB9A54'>**Disartria**</font>

In [13]:
pca_audio = PCA()
pca_audio.fit(np.vstack(data['flatten_log_mel_spectogram']))

# Varianza explicada acumulada
explained_variance_ratio = np.cumsum(pca_audio.explained_variance_ratio_)

# Graficar la varianza explicada acumulada
plt.figure(figsize=(10, 6))
plt.plot(explained_variance_ratio, marker='o', linestyle='--')
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulada')
plt.title('Varianza explicada acumulada por número de componentes principales')
plt.grid(True)
plt.show()

# Imprimir el número de componentes para diferentes niveles de varianza explicada
thresholds = [0.90, 0.95, 0.99]
for threshold in thresholds:
    num_components = np.argmax(explained_variance_ratio >= threshold) + 1
    print(f'Número de componentes para retener el {int(threshold*100)}% de la varianza: {num_components}')

### <font color='#EB9A54'>**Análisis multimodal**</font>

In [14]:
pca_fused_feat = PCA()
pca_fused_feat.fit(np.vstack(data['fused_flattened_features']))

# Varianza explicada acumulada
explained_variance_ratio = np.cumsum(pca_fused_feat.explained_variance_ratio_)

# Graficar la varianza explicada acumulada
plt.figure(figsize=(10, 6))
plt.plot(explained_variance_ratio, marker='o', linestyle='--')
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulada')
plt.title('Varianza explicada acumulada por número de componentes principales')
plt.grid(True)
plt.show()

# Imprimir el número de componentes para diferentes niveles de varianza explicada
thresholds = [0.90, 0.95, 0.99]
for threshold in thresholds:
    num_components = np.argmax(explained_variance_ratio >= threshold) + 1
    print(f'Número de componentes para retener el {int(threshold*100)}% de la varianza: {num_components}')

# <font color='#33e4ff'> **2. Machine Learning 🦾**</color> <a name="tema8">

In [15]:
# Sklearn for datasets.
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold

# Sklearn metrics.
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score)

# Sklearn models
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [16]:
models = [
    ('GNB', GaussianNB()),
    ('DT DEFAULT', DecisionTreeClassifier()),
    ('DT2', DecisionTreeClassifier(max_depth=2)),
    ('DT40', DecisionTreeClassifier(max_depth=40)),
    ('RF50', RandomForestClassifier(n_estimators=50)),
    ('RF100', RandomForestClassifier(n_estimators=100)),
    ('RF200', RandomForestClassifier(n_estimators=200)),
    ('SVM LINEAR', SVC(kernel='linear')),
    ('SVM RBF', SVC(kernel='rbf')),
    ('SVM POLY DEGREE 3', SVC(kernel='poly')),
    ('LR', LogisticRegression(max_iter=2000)),
]

<font color="52F17F">**Escalando las características con sklearn Standard Scaler**</font>

Con esto conseguimos [estandarizar](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) las características eliminando la media (es decir, $\mu = 0$) y escalando a la varianza unitaria (es decir $\sigma = 1$).

<center>
  <img src="https://imgs.search.brave.com/6nO8SCp_NlJdLQRnJRJDgmmNWZjgzK-BvBoRgq6HZfw/rs:fit:860:0:0/g:ce/aHR0cHM6Ly9taXJv/Lm1lZGl1bS5jb20v/bWF4LzE0MDAvMSpH/QWNFajM3c21DT0Na/TXJxcC1yampBLnBu/Zw" width="600" height="400">
</center>

The fit(data) method is used to compute the mean and std dev for a given feature to be used further for scaling. The transform(data) method is used to perform scaling using mean and std dev calculated using the . fit() method. The fit_transform() method does both fits and transform.

<font color="52F17F">**Partición de los datos**</font>

Dado que tenemos 14 pacientes, si hacemos 7 divisiones, cada división intentará mantener aproximadamente el mismo número de pacientes en cada conjunto. Cada división entrena con 12 pacientes y prueba con 2 pacientes. Esto asegura que los datos de un mismo paciente no se mezclen entre los conjuntos de entrenamiento y prueba. [Fuente](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators-for-grouped-data).


In [17]:
# Crear el objeto GroupKFold
group_kfold = GroupKFold(n_splits=7)
X = np.ones((1091, 1))
y = np.ones((1091, 1))

groups = data.index.values

# Realizar la validación cruzada por grupos
for i, (train_index, test_index) in enumerate(group_kfold.split(X, y, groups), 1):
    train_groups = groups[train_index]
    test_groups = groups[test_index]

    print(f"División {i}:")
    print("  Pacientes en entrenamiento:", np.unique(train_groups))
    print("  Pacientes en prueba:", np.unique(test_groups))
    print("  Número de pacientes en entrenamiento:", len(np.unique(train_groups)))
    print("  Número de pacientes en prueba:", len(np.unique(test_groups)))
    print()

<font color="52F17F">**Métricas de evaluación**</font>

En el caso del Parkinson, es más importante saber cuándo mi modelo identifica correctamente a una persona con Parkinson (True Positive), ya que si mi modelo indica que no tiene Parkinson y en realidad sí lo tiene (False Negative), se afecta significativamente su calidad de vida.

<center>
  <img src="https://www.tutorialexample.com/wp-content/uploads/2022/01/how-to-compute-accuracy-precision-recall-and-f1-score-in-machine-learning.png" width="600" height="400">
</center>

Por ende, la métrica que nos interesa es el <font color= "52F17F"> **recall.** </font>




## <font color='#EB9A54'>**2.1 Solo audio features**</font> <a name = "tema10">

In [18]:
X = np.array(data['flatten_log_mel_spectogram'].tolist())
y = data['label'].values

In [19]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [24]:
# Inicializar un diccionario para almacenar las métricas de evaluación
results = {name: {'accuracy': [], 'f1': [], 'precision': [], 'sensitivity': []} for name, _ in models}

# Crear el objeto GroupKFold
group_kfold = GroupKFold(n_splits=7)

groups = data.index.values

'''
Crear un PCA con componentes para el audio.
Este 72 sale de la ejecución de la celda anterior, donde se encuentra
que cantidad de componentes explica la mayor cantidad de varianza de
la imágen
'''
pca = PCA( n_components=72 )


'''
Dentro del for, hacer el fit_transform separado para train y para test,
sólo para el X
'''


# Realizar la validación cruzada por grupos
for (train_index, test_index) in group_kfold.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


    X_train, X_test = pca.fit_transform(X_train), pca.fit_transform(X_test)

    # Entrenar y evaluar cada modelo
    for name, model in models:
        clf = model
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        sensitivity = recall_score(y_test, y_pred)

        results[name]['accuracy'].append(accuracy)
        results[name]['f1'].append(f1)
        results[name]['precision'].append(precision)
        results[name]['sensitivity'].append(sensitivity)

In [25]:
# Show results
for name, metrics in results.items():
    print(f"{11*'='} Resultados para {name} {11*'='}")
    for metric, score in metrics.items():
        print(f"  {metric}: {np.mean(score)*100:.2f}% (+/- {np.std(score)*100:.2f}%)")
    print()

=========== Resultados para GNB ===========
  accuracy: 44.54% (+/- 7.79%)
  f1: 40.34% (+/- 9.29%)
  precision: 43.55% (+/- 8.61%)
  sensitivity: 38.15% (+/- 10.98%)

=========== Resultados para DT DEFAULT ===========
  accuracy: 50.13% (+/- 6.57%)
  f1: 49.90% (+/- 7.28%)
  precision: 50.02% (+/- 6.55%)
  sensitivity: 50.08% (+/- 9.03%)

=========== Resultados para DT2 ===========
  accuracy: 55.18% (+/- 8.79%)
  f1: 46.76% (+/- 17.87%)
  precision: 56.03% (+/- 12.73%)
  sensitivity: 45.94% (+/- 27.30%)

=========== Resultados para DT40 ===========
  accuracy: 50.68% (+/- 7.61%)
  f1: 49.82% (+/- 9.24%)
  precision: 50.40% (+/- 8.20%)
  sensitivity: 49.70% (+/- 11.37%)

=========== Resultados para RF50 ===========
  accuracy: 49.58% (+/- 5.20%)
  f1: 48.62% (+/- 7.38%)
  precision: 49.36% (+/- 5.68%)
  sensitivity: 48.60% (+/- 10.77%)

=========== Resultados para RF100 ===========
  accuracy: 48.76% (+/- 4.20%)
  f1: 48.56% (+/- 9.29%)
  precision: 48.11% (+/- 4.79%)
  sensitivity: 5

## <font color='#EB9A54'>**2.2 Solo video features**</font> <a name = "tema9">

In [28]:
X = np.array(data['flatten_frame_sequence'].tolist())
y = data['label'].values

In [29]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [30]:
# Inicializar un diccionario para almacenar las métricas de evaluación
results = {name: {'accuracy': [], 'f1': [], 'precision': [], 'sensitivity': []} for name, _ in models}

# Crear el objeto GroupKFold
group_kfold = GroupKFold(n_splits=7)

groups = data.index.values

pca = PCA( n_components=69 )

# Realizar la validación cruzada por grupos
for (train_index, test_index) in group_kfold.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


    X_train, X_test = pca.fit_transform(X_train), pca.fit_transform(X_test)

    # Entrenar y evaluar cada modelo
    for name, model in models:
        clf = model
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        sensitivity = recall_score(y_test, y_pred)

        results[name]['accuracy'].append(accuracy)
        results[name]['f1'].append(f1)
        results[name]['precision'].append(precision)
        results[name]['sensitivity'].append(sensitivity)

In [31]:
# Show results
for name, metrics in results.items():
    print(f"{11*'='} Resultados para {name} {11*'='}")
    for metric, score in metrics.items():
        print(f"  {metric}: {np.mean(score)*100:.2f}% (+/- {np.std(score)*100:.2f}%)")
    print()

=========== Resultados para GNB ===========
  accuracy: 59.77% (+/- 13.65%)
  f1: 45.97% (+/- 24.89%)
  precision: 60.75% (+/- 25.59%)
  sensitivity: 39.13% (+/- 24.10%)

=========== Resultados para DT DEFAULT ===========
  accuracy: 55.27% (+/- 15.44%)
  f1: 50.66% (+/- 21.77%)
  precision: 54.31% (+/- 23.98%)
  sensitivity: 48.82% (+/- 21.35%)

=========== Resultados para DT2 ===========
  accuracy: 56.92% (+/- 15.00%)
  f1: 50.12% (+/- 23.04%)
  precision: 55.96% (+/- 19.47%)
  sensitivity: 48.99% (+/- 24.70%)

=========== Resultados para DT40 ===========
  accuracy: 55.00% (+/- 15.53%)
  f1: 50.54% (+/- 21.75%)
  precision: 54.04% (+/- 23.99%)
  sensitivity: 48.82% (+/- 21.35%)

=========== Resultados para RF50 ===========
  accuracy: 46.29% (+/- 13.60%)
  f1: 40.64% (+/- 15.69%)
  precision: 47.44% (+/- 21.82%)
  sensitivity: 38.01% (+/- 15.97%)

=========== Resultados para RF100 ===========
  accuracy: 46.75% (+/- 13.61%)
  f1: 39.47% (+/- 15.00%)
  precision: 47.87% (+/- 22.27%)

## <font color='#EB9A54'>**2.3 Fusion of features**</font> <a name = "tema9">

In [32]:
X = np.array(data['fused_flattened_features'].tolist())
y = data['label'].values

In [33]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [34]:
# Inicializar un diccionario para almacenar las métricas de evaluación
results = {name: {'accuracy': [], 'f1': [], 'precision': [], 'sensitivity': []} for name, _ in models}

# Crear el objeto GroupKFold
group_kfold = GroupKFold(n_splits=7)

groups = data.index.values

pca = PCA(n_components=91)

# Realizar la validación cruzada por grupos
for (train_index, test_index) in group_kfold.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train, X_test = pca.fit_transform(X_train), pca.fit_transform(X_test)

    # Entrenar y evaluar cada modelo
    for name, model in models:
        clf = model
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        sensitivity = recall_score(y_test, y_pred)

        results[name]['accuracy'].append(accuracy)
        results[name]['f1'].append(f1)
        results[name]['precision'].append(precision)
        results[name]['sensitivity'].append(sensitivity)

In [35]:
# Show results
for name, metrics in results.items():
    print(f"{11*'='} Resultados para {name} {11*'='}")
    for metric, score in metrics.items():
        print(f"  {metric}: {np.mean(score)*100:.2f}% (+/- {np.std(score)*100:.2f}%)")
    print()

=========== Resultados para GNB ===========
  accuracy: 63.90% (+/- 16.74%)
  f1: 49.64% (+/- 30.56%)
  precision: 63.18% (+/- 26.85%)
  sensitivity: 45.58% (+/- 33.02%)

=========== Resultados para DT DEFAULT ===========
  accuracy: 54.82% (+/- 15.86%)
  f1: 49.98% (+/- 21.52%)
  precision: 54.42% (+/- 24.25%)
  sensitivity: 47.53% (+/- 20.83%)

=========== Resultados para DT2 ===========
  accuracy: 57.19% (+/- 14.80%)
  f1: 50.32% (+/- 23.00%)
  precision: 56.51% (+/- 18.83%)
  sensitivity: 49.17% (+/- 24.67%)

=========== Resultados para DT40 ===========
  accuracy: 55.00% (+/- 15.88%)
  f1: 50.18% (+/- 21.59%)
  precision: 54.55% (+/- 24.31%)
  sensitivity: 47.72% (+/- 20.71%)

=========== Resultados para RF50 ===========
  accuracy: 47.67% (+/- 14.15%)
  f1: 43.79% (+/- 16.09%)
  precision: 48.13% (+/- 19.41%)
  sensitivity: 41.31% (+/- 15.57%)

=========== Resultados para RF100 ===========
  accuracy: 48.86% (+/- 13.40%)
  f1: 42.34% (+/- 16.91%)
  precision: 48.91% (+/- 20.72%)

# <font color='#1f8bff'> **3. Bibliografía**</font>


* Mangale, S. (2021, 15 diciembre). Scree Plot - SANCHITA MANGALE - Medium. Medium. https://sanchitamangale12.medium.com/scree-plot-733ed72c8608
* PCA. (s. f.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
* GroupKFold. (s. f.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html
* StandardScaler. (s. f.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html








